
# Laboratorio: Predicción de Churn de Clientes (Telecom)

Este notebook implementa un mini pipeline de **minería de datos** para predecir la deserción (churn) de clientes en una empresa de telecomunicaciones, usando el archivo `clientes_telecom.csv`.

Pasos:

1. Carga del dataset  
2. Exploración y análisis descriptivo  
3. Preparación de datos  
4. Entrenamiento de un modelo de clasificación  
5. Evaluación del modelo  
6. Interpretación de resultados de negocio  


## 1. Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Configuración opcional de gráficos
plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['figure.dpi'] = 100
sns.set(style="whitegrid")

## 2. Cargar el dataset

Antes de ejecutar esta celda, sube el archivo `clientes_telecom.csv` a tu entorno de ejecución (por ejemplo, en Google Colab, usando la sección **Files** → **Upload**).

In [ ]:
# Cambia la ruta si es necesario según tu entorno
ruta_archivo = "/content/clientes_telecom.csv"

df = pd.read_csv(ruta_archivo)
df.head()

## 3. Exploración inicial del dataset

In [ ]:
print("Dimensiones del dataset (filas, columnas):", df.shape)
print("\nInformación general:")
print(df.info())

In [ ]:
print("Descripción estadística:")
df.describe()

In [ ]:
print("Proporción de churn (0 = se queda, 1 = se va):")
df['churn'].value_counts(normalize=True)

## 4. Análisis de variables clave

### 4.1 Churn según uso de la app móvil

In [ ]:
churn_por_app = df.groupby('usa_app_movil')['churn'].mean()
print(churn_por_app)

plt.figure()
churn_por_app.plot(kind='bar')
plt.xticks([0,1], ['No usa app', 'Sí usa app'], rotation=0)
plt.ylabel('Tasa de churn')
plt.title('Churn según uso de app móvil')
plt.show()

### 4.2 Churn según número de reclamos

In [ ]:
churn_por_reclamos = df.groupby('reclamos_ultimo_anio')['churn'].mean()
print(churn_por_reclamos)

plt.figure()
churn_por_reclamos.plot(kind='bar')
plt.xlabel('Reclamos último año')
plt.ylabel('Tasa de churn')
plt.title('Churn según número de reclamos')
plt.show()

## 5. Preparación de los datos (train/test)

In [ ]:
# Variables predictoras (X) y variable objetivo (y)
X = df[['edad', 'ingresos_mensuales', 'antiguedad_meses',
        'lineas_contratadas', 'usa_app_movil', 'reclamos_ultimo_anio']]
y = df['churn']

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape

## 6. Entrenamiento del modelo (Regresión Logística)

In [ ]:
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

## 7. Evaluación del modelo

In [ ]:
acc = accuracy_score(y_test, y_pred)
print(f"Exactitud del modelo: {acc:.2f}")

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", cm)

plt.figure()
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicción")
plt.ylabel("Valor real")
plt.title("Matriz de confusión - Modelo de churn")
plt.show()

In [ ]:
print("Reporte de clasificación:\n")
print(classification_report(y_test, y_pred))

## 8. Importancia de variables

In [ ]:
coef = pd.DataFrame({
    "variable": X.columns,
    "coeficiente": modelo.coef_[0]
}).sort_values("coeficiente", ascending=False)

coef

In [ ]:
plt.figure()
plt.bar(coef['variable'], coef['coeficiente'])
plt.axhline(0, linestyle='--')
plt.xticks(rotation=45)
plt.title("Importancia de variables en el modelo de churn")
plt.ylabel("Coeficiente (Regresión Logística)")
plt.show()

## 9. Conclusiones

- El modelo de regresión logística permite estimar la probabilidad de churn de cada cliente.
- Variables como los reclamos, el uso (o no) de la app móvil y la antigüedad influyen en la probabilidad de que un cliente se vaya.
- Este tipo de análisis ayuda al negocio a enfocar campañas de retención en los clientes con mayor riesgo de churn.
